### TAO remote client - Segmentation

Transfer learning is the process of transferring learned features from one application to another. It is a commonly used training technique where you use a model trained on one task and re-train to use it on a different task. Train Adapt Optimize (TAO) Toolkit  is a simple and easy-to-use Python based AI toolkit for taking purpose-built AI models and customizing them with users' own data.

![image](https://d29g4g2dyqv443.cloudfront.net/sites/default/files/akamai/TAO/tlt-tao-toolkit-bring-your-own-model-diagram.png)

### Sample prediction for an Semantic Segmentation model - Segformer
<img align="center" src="../example_images/sample_semantic_segmentation.jpg">

### The workflow in a nutshell

- Pulling datasets from cloud
- Running dataset convert (For Mask-RCNN)
- Getting a PTM from NGC
- Model Actions
    - Train (Normal/AutoML)
    - Evaluate
    - Prune, retrain
    - Export
    - TAO-Deploy
    - Inference on TAO, TRT
    - Delete experiments/dataset

### Table of contents

1. [Install TAO remote client ](#head-1)
1. [FIXME's](#head-2)
1. [Login](#head-3)
1. [Create a cloud workspace](#head-2)
1. [Set dataset formats](#head-4)
1. [Create and pull train dataset](#head-5)
1. [Create and pull val dataset](#head-6)
1. [List the created datasets](#head-7)
1. [Train Dataset convert action](#head-8) (for specific models)
1. [Val dataset convert action](#head-9) (for specific models)
1. [Create experiment (via create-job)](#head-10)
1. [List experiments](#head-11)
1. [Assign train, eval datasets](#head-12)
1. [Assign PTM](#head-13)
1. [Set AutoML related configurations](#head-14)
1. [Train](#head-15)
1. [View hyperparameters that are enabled by default](#head-15.1)
1. [Evaluate](#head-16)
1. [Optimize: Prune, retrain and evaluate](#head-17)
1. [Export](#head-18)
1. [TRT Engine generation using TAO-Deploy](#head-19)
1. [TAO inference](#head-20)
1. [TRT inference](#head-21)
1. [Delete experiment](#head-22)
1. [Delete dataset](#head-23)

### Requirements
Please find the server requirements [here](https://docs.nvidia.com/tao/tao-toolkit/text/tao_toolkit_api/api_setup.html#)

### Install TAO remote client <a class="anchor" id="head-1"></a>

In [ ]:
# SKIP this step IF you have already installed the TAO-Client wheel.
! pip3 install nvidia-tao-client

In [ ]:
# View the version of the TAO-Client
! tao --version

### Import python packages required for notebook <a class="anchor" id="head-1"></a>

In [ ]:
import os
import subprocess
import json
import time
from IPython.display import clear_output

In [ ]:
# Restore variable in case of jupyter session restart and resume execution where it left off
%store -r model_name
%store -r automl_enabled
%store -r automl_algorithm
%store -r workspace_id
%store -r train_dataset_id
%store -r test_dataset_id
%store -r experiment_id
%store -r job_map

In [ ]:
namespace = 'default'
job_map = {}

### To see the dataset folder structure required for the models supported in this notebook, visit the notebooks under dataset_prepare like for [this notebook](../dataset_prepare/segmentation.ipynb)

### FIXME's <a class="anchor" id="head-2"></a>

1. Assign a model_name in FIXME 1
1. (Optional) Enable AutoML if needed in FIXME 2
1. (Optional) Choose between bayesian and hyperband automl_algorithm in FIXME 3 (If automl was enabled in FIXME2)
1. Assign a workdir in FIXME 4 for log file download
1. Assign the ip_address and port_number in FIXME 5 ([info](https://docs.nvidia.com/tao/tao-toolkit/text/tao_toolkit_api/api_rest_api.html))
1. Assign the ngc_key variable in FIXME 6
1. Assign the ngc_org_name variable in FIXME 7
1. Set cloud storage details in FIXME 8
1. Assign path of datasets relative to the bucket in FIXME 9
1. Database backup/restore archive filename in FIXME 10

#### Choose a segmentation model

In [ ]:
# Available models (#FIXME 1):
# 1. segformer - https://docs.nvidia.com/tao/tao-toolkit/text/cv_finetuning/pytorch/segformer.html
# 2. mask2former - https://docs.nvidia.com/tao/tao-toolkit/text/cv_finetuning/pytorch/instance_segmentation/mask2former.html
# 3. mask_grounding_dino - https://docs.nvidia.com/tao/tao-toolkit/text/cv_finetuning/pytorch/instance_segmentation/mask_grounding_dino.html
# 4. oneformer - https://docs.nvidia.com/tao/tao-toolkit/text/cv_finetuning/pytorch/instance_segmentation/oneformer.html

# FIXME 1 (Add the model name from the above mentioned list)
os.environ["TAO_MODEL_NAME"] = model_name = os.environ.get("TAO_MODEL_NAME", "segformer")
%store model_name

#### Toggle AutoML params
[AutoML documentation](https://docs.nvidia.com/tao/tao-toolkit/text/automl/automl.html#getting-started)

In [ ]:
# FIXME 2: Set to True if you want to run automl for the model chosen in the previous cell
automl_enabled = os.environ.get("TAO_AUTOML_ENABLED", "False").lower() == "true"
os.environ["TAO_AUTOML_ENABLED"] = str(automl_enabled)
# FIXME 3: One of bayesian/hyperband
os.environ["TAO_AUTOML_ALGORITHM"] = automl_algorithm = os.environ.get("TAO_AUTOML_ALGORITHM", "bayesian")

%store automl_enabled
%store automl_algorithm

### Common Functions used across the notebook

#### Function to parse logs

In [ ]:
def my_tail(model_name_cli, job_id):
	status = None
	while True:
		time.sleep(10)
		clear_output(wait=True)
		response = subprocess.getoutput(f"tao {model_name_cli} get-job-metadata --job-id {job_id}")
		response = json.loads(response)
		if response and "status" in response.keys() and response.get("status") in ("Done", "Error", "Canceled", "Paused"):
			print(json.dumps(response.get("job_details", {}), indent=4))
			status = response.get("status")
			assert status == "Done", f"Status is not Done, it is {status}"
			break

		logs = subprocess.getoutput(f"tao {model_name_cli} get-job-logs --job-id {job_id}")
		if not logs:
			continue
		log_content_lines = logs.split("\n")        
		for line in log_content_lines:
			print(line.strip())
			if line.strip() == "Error EOF":
				status = "Error"
				break
			elif line.strip() == "Done EOF":
				status = "Done"
				break
		if status is not None:
			break
	return status

#### Function to load login details from saved config

In [ ]:
def load_tao_credentials_from_config():
    """Load TAO credentials from ~/.tao/config and set as environment variables"""
    from configparser import ConfigParser
    from pathlib import Path
    import os
    
    config_path = Path.home() / '.tao' / 'config'
    
    if not config_path.exists():
        print(f"Warning: Config file not found at {config_path}")
        print("Please run 'tao login' first")
        return False
    
    try:
        parser = ConfigParser()
        parser.read(config_path)
        
        # Read from [CURRENT] section
        if parser.has_section('CURRENT'):
            section = parser['CURRENT']
        else:
            print("Warning: No [CURRENT] section found in config file")
            return False
        
        # Set environment variables
        if 'tao_base_url' in section:
            os.environ['TAO_BASE_URL'] = section['tao_base_url']
            print(f"✓ TAO_BASE_URL set to: {section['tao_base_url']}")
        
        if 'tao_org' in section:
            os.environ['TAO_ORG'] = section['tao_org']
            print(f"✓ TAO_ORG set to: {section['tao_org']}")
        
        if 'tao_token' in section:
            os.environ['TAO_TOKEN'] = section['tao_token']
            print(f"✓ TAO_TOKEN set (expires: check token if auth fails)")
        
        return True
        
    except Exception as e:
        print(f"Error reading config file: {e}")
        return False

#### Set API service's host information

In [ ]:
# FIXME 4: Set TAO API environment variables

# Set to your TAO API endpoint
os.environ["TAO_BASE_URL"] = os.environ.get("TAO_BASE_URL", "https://your_tao_ip_address:port/api/v2")

#### Set NGC Personal key for authentication and NGC org to access API services

In [ ]:
os.environ["NGC_KEY"] = ngc_key = os.environ.get("NGC_KEY", "your_ngc_key")  # FIXME6 example: (Add NGC Personal key)
os.environ["HUGGINGFACE_TOKEN"] = huggingface_token = os.environ.get("HUGGINGFACE_TOKEN", "") # FIXME6 example: (Add HuggingFace token - required for oneformer)

In [ ]:
os.environ["NGC_ORG"] = ngc_org_name = os.environ.get("NGC_ORG", "nvstaging")  # FIXME7 your NGC ORG

### Login <a class="anchor" id="head-3"></a>

In [ ]:
# Exchange NGC_API_KEY for JWT
! tao login --ngc-org-name {ngc_org_name} --ngc-key {ngc_key} --enable-telemetry

# Load credentials when this cell runs
load_tao_credentials_from_config()

### Get NVCF gpu details <a class="anchor" id="head-2"></a>

 One of the keys of the response json are to be used as platform_id when you run each job

In [ ]:
# # Valid only for NVCF backend during TAO-API helm deployment currently
# # response = json.loads(subprocess.getoutput(f'tao get-gpu-types'))
# print((json.dumps(response, indent=4)))

### Create cloud workspace
This workspace will be the place where your datasets reside and your results of TAO API jobs will be pushed to.

If you want to have different workspaces for dataset and experiment, duplocate the workspace creation part and adjust the metadata accordingly.

In [ ]:
# FIXME 7: Dataset Cloud bucket details to download dataset or push job artifacts for jobs

cloud_metadata = {}

# A Representative name for this cloud info
os.environ["TAO_WORKSPACE_NAME"] = cloud_metadata["name"] = os.environ.get("TAO_WORKSPACE_NAME", "AWS workspace info")

# Cloud specific details. Below is assuming AWS.
cloud_metadata["cloud_specific_details"] = {}

 # Whether it is AWS, HuggingFace or Azure
os.environ["TAO_WORKSPACE_CLOUD_TYPE"] = cloud_metadata["cloud_specific_details"]["cloud_type"] = os.environ.get("TAO_WORKSPACE_CLOUD_TYPE", "aws")

# Bucket region
os.environ["TAO_WORKSPACE_CLOUD_REGION"] = cloud_metadata["cloud_specific_details"]["cloud_region"] = os.environ.get("TAO_WORKSPACE_CLOUD_REGION", "us-west-1")

# Bucket name
os.environ["TAO_WORKSPACE_CLOUD_BUCKET_NAME"] = cloud_metadata["cloud_specific_details"]["cloud_bucket_name"] = os.environ.get("TAO_WORKSPACE_CLOUD_BUCKET_NAME", "bucket_name")

# Access and Secret keys
os.environ["TAO_WORKSPACE_CLOUD_ACCESS_KEY"] = cloud_metadata["cloud_specific_details"]["access_key"] = os.environ.get("TAO_WORKSPACE_CLOUD_ACCESS_KEY", "access_key")
os.environ["TAO_WORKSPACE_CLOUD_SECRET_KEY"] = cloud_metadata["cloud_specific_details"]["secret_key"] = os.environ.get("TAO_WORKSPACE_CLOUD_SECRET_KEY", "secret_key")

In [ ]:
workspace_id = subprocess.getoutput(f"tao {model_name} create-workspace --name 'AWS Workspace' --cloud-type {cloud_metadata["cloud_specific_details"]["cloud_type"]} --cloud-specific-details '{json.dumps(cloud_metadata["cloud_specific_details"])}'")
print(workspace_id)
%store workspace_id

In [ ]:
# #Optional: Restore database with a mongodump file saved in workspace dump/archive/{backup_filename}
# backup_file_name = "mongodump.tar.gz" # FIXME 10
# response = subprocess.getoutput(f"tao {model_name} restore-workspace --workspace-id {workspace_id} --backup_file_name {backup_file_name}")
# print(response)

#### Set dataset path (path within cloud bucket)

In [ ]:
# FIXME 8 : Set paths relative to cloud bucket
os.environ["TAO_TRAIN_DATASET_PATH"] = train_dataset_path =  os.environ.get("TAO_TRAIN_DATASET_PATH", f"/data/segmentation_{model_name}_train")
os.environ["TAO_EVAL_DATASET_PATH"] = eval_dataset_path = os.environ.get("TAO_EVAL_DATASET_PATH", f"/data/segmentation_{model_name}_val")

### Set dataset formats <a class="anchor" id="head-3"></a>

In [ ]:
# Create train dataset
ds_type = "segmentation"
if model_name == "segformer":
    ds_format = "unet"
elif model_name in ["mask2former", "oneformer"]:
    ds_format = "coco_panoptic"

### Create and pull train dataset <a class="anchor" id="head-5"></a>

In [ ]:
# Path of file within bucket
if model_name == "mask_grounding_dino":
    ds_format = "odvg"
train_dataset_id = subprocess.getoutput(f"tao {model_name} create-dataset --dataset-type {ds_type} --dataset-format {ds_format} --workspace-id {workspace_id} --cloud-file-path {train_dataset_path} --use-for '{json.dumps(['training'])}'")
print(train_dataset_id)
%store train_dataset_id

In [ ]:
# Check progress
while True:
    clear_output(wait=True)
    response = subprocess.getoutput(f"tao {model_name} get-dataset-metadata --dataset-id {train_dataset_id} ")
    try:
        response = json.loads(response)
    except Exception as e:
        print(response)
        raise e
    print(json.dumps(response, sort_keys=True, indent=4))
    if response.get("status") == "invalid_pull":
        raise ValueError("Dataset pull failed")
    if response.get("status") == "pull_complete":
        break
    time.sleep(5)

#### Uncomment if you want to remove corrupted images in your dataset

In [ ]:
# # This packages data-services experiments create and running the job of removing corrupted images
# from remove_corrupted_images import remove_corrupted_images_workflow
# # try:
#     from remove_corrupted_images import remove_corrupted_images_workflow
#     train_dataset_id = remove_corrupted_images_workflow(workspace_id, train_dataset_id)
# except Exception as e:
#     raise e

### Create and pull val dataset <a class="anchor" id="head-6"></a>

In [ ]:
if model_name == "mask_grounding_dino":
    ds_format = "coco"
eval_dataset_id = subprocess.getoutput(f"tao {model_name} create-dataset --dataset-type {ds_type} --dataset-format {ds_format} --workspace-id {workspace_id} --cloud-file-path {eval_dataset_path} --use-for '{json.dumps(['testing'])}'")
print(eval_dataset_id)
%store eval_dataset_id

In [ ]:
# Check progress
while True:
    clear_output(wait=True)
    response = subprocess.getoutput(f"tao {model_name} get-dataset-metadata --dataset-id {eval_dataset_id} ")
    try:
        response = json.loads(response)
    except Exception as e:
        print(response)
        raise e
    print(json.dumps(response, sort_keys=True, indent=4))
    if response.get("status") == "invalid_pull":
        raise ValueError("Dataset pull failed")
    if response.get("status") == "pull_complete":
        break
    time.sleep(5)

#### Uncomment if you want to remove corrupted images in your dataset

In [ ]:
# # This packages data-services experiments create and running the job of removing corrupted images
# from remove_corrupted_images import remove_corrupted_images_workflow
# # try:
#     from remove_corrupted_images import remove_corrupted_images_workflow
#     test_dataset_id = remove_corrupted_images_workflow(workspace_id, test_dataset_id)
# except Exception as e:
#     raise e

### List the created datasets <a class="anchor" id="head-7"></a>

In [ ]:
message = subprocess.getoutput(f"tao {model_name} list-datasets")
message = json.loads(message)
for rsp in message:
    rsp_keys = rsp.keys()
    assert "id" in rsp_keys
    assert "type" in rsp_keys
    assert "format" in rsp_keys
    assert "name" in rsp_keys
    print(rsp["id"],"\t",rsp["type"],"\t",rsp["format"],"\t\t",rsp["name"])

### Set common params across all jobs <a class="anchor" id="head-9"></a>

In [ ]:
# These parameters are common to all jobs and will be used when creating the actual job:
encode_key = "tlt_encode"
if model_name == "segformer":
    encode_key = "nvidia_tao"
docker_env_vars = {"HF_TOKEN": huggingface_token}

### Assign PTM <a class="anchor" id="head-13"></a>

Search for PTM on NGC for the Segmentation model chosen

In [ ]:
# List base experiments (PTMs) using TAO SDK  
filter_params = {"network_arch": model_name}
message = subprocess.getoutput(f"tao {model_name} list-base-experiments --filter-params '{json.dumps(filter_params)}'")

message = json.loads(message)
# Store base experiments list for reuse
base_experiments = message

print(f" Available base experiments (PTMs) for {model_name}:")
print("name\t\t\t     model id\t\t\t     network architecture")
print("-" * 120)

for exp in base_experiments:
    exp_name = exp.get("name", "N/A")
    exp_id = exp.get("id", "N/A")
    exp_arch = exp.get("network_arch", "N/A")
    print(f"{exp_name}\t{exp_id}\t{exp_arch}")

In [ ]:
# Assigning pretrained models to different networks
# From the output of previous cell make the appropriate changes to this map if you want to change the default PTM backbone.
# Changing the default backbone here requires changing default spec/config during train/eval etc like for example
# If you are changing the ptm to resnet34, then you have to modify the config key num_layers if it exists to 34 manually
pretrained_map = {"segformer" : "pretrained_segformer_imagenet:fan_hybrid_small",
                  "mask2former": "mask2former:mask2former_swint_trainable_v1.0",
                  "mask_grounding_dino": "mask_grounding_dino:mask_grounding_dino_swin_tiny_commercial_trainable_v1.0",
                  "oneformer": "oneformer:oneformer_coco_pretrained_research"
                  }
no_ptm_models = set([])

In [ ]:
# Get pretrained model using TAO SDK
selected_ptm_id = None
if model_name not in no_ptm_models:

    # Search for PTM with given NGC path
    for exp in base_experiments:
        ngc_path = exp.get("ngc_path", "")
        if ngc_path.endswith(pretrained_map[model_name]):
            selected_ptm_id = exp.get("id")
            print(" Selected PTM metadata:")
            print(json.dumps(exp, indent=4))
            break

    if not selected_ptm_id:
        print(f" PTM with NGC path ending in '{pretrained_map[model_name]}' not found!")

if model_name not in no_ptm_models and selected_ptm_id:
    print(f" PTM ID {selected_ptm_id} will be used as base_experiment_id in job creation")
else:
    print(" No PTM will be used (training from scratch)")

### Train <a class="anchor" id="head-15"></a>

#### View hyperparameters that are enabled for AutoML by default <a class="anchor" id="head-14"></a>

In [ ]:
if automl_enabled:
    # View default automl params enabled
    automl_params = subprocess.getoutput(f"tao {model_name} get-automl-defaults")
    print(automl_params)
    automl_params = json.loads(automl_params)

#### Set AutoML related configurations <a class="anchor" id="head-15.1"></a>
Refer to these hyper-links to see the parameters supported by each network and add more parameters if necessary in addition to the default automl enabled parameters:

[Segformer](https://github.com/NVIDIA/tao_front_end_services/tree/main/api/specs_utils/specs/segformer/segformer%20-%20train.csv)

In [ ]:
# Prepare AutoML configuration if enabled (matching SDK approach)
automl_information = None

if automl_enabled:
    # Choose any metric that is present in the kpi dictionary present in the model's status.json
    metric = "kpi"
    
    automl_information = {
        "automl_enabled": True,
        "automl_algorithm": automl_algorithm,
        "automl_max_recommendations": 20,  # Only for bayesian
        "automl_R": 27,  # Only for hyperband
        "automl_nu": 3,  # Only for hyperband
        "epoch_multiplier": 1,  # Only for hyperband
        "override_automl_disabled_params": False,
        "automl_hyperparameters": str(automl_params),
        "metric": metric
    }
    
    print(" AutoML configuration prepared for job creation:")
    print(json.dumps(automl_information, sort_keys=True, indent=4))
else:
    print(" AutoML is disabled - training will use standard approach")

#### Provide train specs

In [ ]:
# Default train model specs
train_specs_response = subprocess.getoutput(f"tao {model_name} get-job-schema --action train")
train_specs_schema = json.loads(train_specs_response)
train_specs = train_specs_schema.get("default", {})
print(json.dumps(train_specs, indent=4))

In [ ]:
# Customize train model specs
train_specs["train"]["num_gpus"] = 1
train_specs["train"]["num_epochs"] = 10
train_specs["train"]["checkpoint_interval"] = 10
train_specs["train"]["validation_interval"] = 10
# train_specs["model"]["backbone"]["type"] = "fan_tiny_8_p4_hybrid"
if model_name == "segformer":
    train_specs["dataset"]["segment"]["batch_size"] = 4
elif model_name == "mask2former":
    train_specs["model"]["sem_seg_head"]["num_classes"] = 90
    train_specs["dataset"]["contiguous_id"] = True
elif model_name == "oneformer":
    train_specs["model"]["sem_seg_head"]["num_classes"] = 133
    train_specs["dataset"]["contiguous_id"] = True
    train_specs["train"]["precision"] = "32"
elif model_name == "mask_grounding_dino":
    train_specs["dataset"]["val_data_sources"]["data_type"] = "OD"
    # # for v1.0
    # if "rela" in train_specs["model"]["loss_types"]:
    #     train_specs ["model"]["loss_types"] = [loss_type for loss_type in train_specs["model"]["loss_types"] if loss_type != "rela"]
    # train_specs["model"]["num_region_queries"] = 0
    # for v2.0
    if "rela" not in train_specs["model"]["loss_types"]:
        train_specs["model"]["loss_types"].append("rela")
    train_specs["model"]["num_region_queries"] = 100
print(json.dumps(train_specs, sort_keys=True, indent=4))

#### Run train

In [ ]:
# Add --platform_id uuid for NVCF backend, where the uuid is a key from output of tao gpu-types
train_datasets_json = json.dumps([train_dataset_id])
automl_settings = json.dumps(automl_information) if automl_information else 'null'
job_id = subprocess.getoutput(f"tao {model_name} create-job --kind experiment --action train --name {model_name}_training_job --encryption-key {encode_key} --workspace-id {workspace_id} --docker-env-vars '{json.dumps(docker_env_vars)}' --base-experiment-ids {selected_ptm_id} --train-datasets '{train_datasets_json}' --eval-dataset {eval_dataset_id} --calibration-dataset {train_dataset_id} --specs '{json.dumps(train_specs)}' --automl-settings '{automl_settings}'")
job_map["train_" + model_name] = job_id
print(job_id)
%store job_map

In [ ]:
# Monitor job status
job_id = job_map["train_" + model_name]
if automl_enabled:    
    while True:
        clear_output(wait=True)
        response = subprocess.getoutput(f"tao {model_name} get-job-metadata --job-id {job_id}")
        response = json.loads(response)
        job_details = response.get("job_details", {})
        if "error_desc" in response.keys() and response["error_desc"] in ("Job trying to retrieve not found", "No AutoML run found"):
            print("Job is being created")
            time.sleep(5)
            continue
        print(json.dumps(job_details, indent=4))
        assert "status" in response.keys() and response.get("status") != "Error"
        if response.get("status") in ["Done","Error"]:
            break
        time.sleep(15)
else:
    # Check status (the file won't exist until the backend Toolkit container is running -- can take several minutes)
    status = my_tail(model_name, job_id)

In [ ]:
## To Stop an AutoML JOB
#    1. Stop the 'Monitor job status' cell (the cell right before this cell) manually
#    2. Uncomment the snippet in the next cell and run the cell

In [ ]:
# if automl_enabled:
#     job_id = job_map["train_" + model_name]
#     job_id = subprocess.getoutput(f"tao {model_name} pause-job --job-id {job_id}")
#     job_map["canceled_" + model_name] = job_id
#     print(job_id)
#     %store job_map

In [ ]:
## Resume AutoML

In [ ]:
# Uncomment the below snippet if you want to resume an already stopped AutoML job and then run the 'Monitor job status' cell above (4th cell above from this cell)
# if automl_enabled:
#     job_id = job_map["train_" + model_name]
#     job_id = subprocess.getoutput(f"tao {model_name} resume-job --job-id {job_id} --specs '{json.dumps(train_specs)}' {ds_convert_parent}")
#     job_map["resumed_" + model_name] = job_id
#     print(job_id)
#     %store job_map

### Publish model

#### Edit the method of choosing checkpoint from list of train checkpoint files

In [ ]:
# Print model handler parameters
job_id = job_map["train_" + model_name]
model_parameters = subprocess.getoutput(f"tao {model_name} get-job-metadata --job-id {job_id}")
model_parameters = json.loads(model_parameters)
update_checkpoint_choosing = {}
update_checkpoint_choosing["checkpoint_choose_method"] = model_parameters["checkpoint_choose_method"]
update_checkpoint_choosing["checkpoint_epoch_number"] = model_parameters["checkpoint_epoch_number"]
print(json.dumps(update_checkpoint_choosing, indent=4))

In [ ]:
# Checkpoint method configuration
# Checkpoint selection is handled per-job, not per-experiment
# You can configure this when creating export/inference jobs if needed

# Example: Change checkpoint selection method for future jobs
update_checkpoint_choosing["checkpoint_choose_method"] = "latest_model"  # Choose between best_model/latest_model/from_epoch_number
# Note: If from_epoch_number is chosen, you would specify the epoch in job creation specs

print("Checkpoint selection configuration updated:")
print(f"Method: {update_checkpoint_choosing['checkpoint_choose_method']}")
print("This will be applied to future job creations")
print(json.dumps(update_checkpoint_choosing, sort_keys=True, indent=4))

json_update_data = json.dumps(update_checkpoint_choosing)
updated_job_data = subprocess.getoutput(f"tao {model_name} update-job --job-id {job_id} --update-data '{json_update_data}'")
print("\n Updated job data:")
print(json.dumps(json.loads(updated_job_data), indent=4))

#### Push model to private ngc team registry

In [ ]:
display_name = f"TAO {model_name}"  # Display name for the model to be published on the model card
description = f"Train {model_name}"  # Short description for the model to be published on the model card
team = "tao"  # Team within org for the model to be published to

job_id = job_map["train_" + model_name]
message = subprocess.getoutput(f"tao {model_name} publish-model --job-id {job_id} --display-name='{display_name}' --description='{description}' --team {team}")
print(message)

#### Remove model from private ngc team registry

In [ ]:
# message = subprocess.getoutput(f"tao {model_name} remove-published-model --job-id {job_id} --team {team}")
# print(message)

### Evaluate <a class="anchor" id="head-16"></a>

#### Provide evaluate specs

In [ ]:
# Default evaluate model specs
eval_specs_response = subprocess.getoutput(f"tao {model_name} get-job-schema --action evaluate")
eval_specs_schema = json.loads(eval_specs_response)
eval_specs = eval_specs_schema.get("default", {})
print(json.dumps(eval_specs, indent=4))

In [ ]:
# Apply changes to the specs if required
if model_name == "segformer":
    eval_specs["evaluate"]["batch_size"] = 4
elif model_name == "mask2former":
    eval_specs["model"]["sem_seg_head"]["num_classes"] = 90
    eval_specs["dataset"]["contiguous_id"] = True
elif model_name == "oneformer":
    eval_specs["model"]["sem_seg_head"]["num_classes"] = 133
elif model_name == "mask_grounding_dino":
    eval_specs["dataset"]["test_data_sources"]["data_type"] = "OD"
print(json.dumps(eval_specs, sort_keys=True, indent=4))

#### Run evaluate

In [ ]:
# Add --platform_id uuid for NVCF backend, where the uuid is a key from output of tao gpu-types
parent = job_map["train_" + model_name]
train_datasets_json = json.dumps([train_dataset_id])
job_id = subprocess.getoutput(f"tao {model_name} create-job --kind experiment --action evaluate --name {model_name}_evaluation_job --encryption-key {encode_key} --workspace-id {workspace_id} --docker-env-vars '{json.dumps(docker_env_vars)}' --train-datasets '{train_datasets_json}' --eval-dataset {eval_dataset_id} --parent-job-id {parent} --specs '{json.dumps(eval_specs)}'")
job_map["eval_" + model_name] = job_id
print(job_id)
%store job_map

In [ ]:
# Check status (the file won't exist until the backend Toolkit container is running -- can take several minutes)
job_id = job_map["eval_" + model_name]
status = my_tail(model_name, job_id)

### Export <a class="anchor" id="head-18"></a>

#### Provide export specs

In [ ]:
# Default export model specs
export_specs_response = subprocess.getoutput(f"tao {model_name} get-job-schema --action export")
export_specs_schema = json.loads(export_specs_response)
export_specs = export_specs_schema.get("default", {})
print(json.dumps(export_specs, indent=4))

In [ ]:
# Customize export model specs
# Apply changes to the specs dictionary here if required
if model_name == "mask2former":
    export_specs["model"]["sem_seg_head"]["num_classes"] = 90
elif model_name == "oneformer":
    export_specs["model"]["sem_seg_head"]["num_classes"] = 133
    export_specs["model"]["export"] = True
print(json.dumps(export_specs, indent=4))

#### Run export

In [ ]:
# Add --platform_id uuid for NVCF backend, where the uuid is a key from output of tao gpu-types
parent = job_map["train_" + model_name]
job_id = subprocess.getoutput(f"tao {model_name} create-job --kind experiment --action export --name {model_name}_export_job --encryption-key {encode_key} --workspace-id {workspace_id} --docker-env-vars '{json.dumps(docker_env_vars)}' --parent-job-id {parent} --base-experiment-ids {selected_ptm_id} --specs '{json.dumps(export_specs)}'")
job_map["export_" + model_name] = job_id
print(job_id)
%store job_map

In [ ]:
# Check status (the file won't exist until the backend Toolkit container is running -- can take several minutes)
job_id = job_map["export_" + model_name]
status = my_tail(model_name, job_id)

### TRT Engine generation using TAO-Deploy <a class="anchor" id="head-19"></a>

#### Provide trt engine generation specs

In [ ]:
# Default gen_trt_engine model specs
tao_deploy_specs_response = subprocess.getoutput(f"tao {model_name} get-job-schema --action gen_trt_engine")
tao_deploy_specs_schema = json.loads(tao_deploy_specs_response)
tao_deploy_specs = tao_deploy_specs_schema.get("default", {})
print(json.dumps(tao_deploy_specs, indent=4))

In [ ]:
# Customize gen_trt_engine model specs
if model_name == "segformer":
    tao_deploy_specs["gen_trt_engine"]["tensorrt"]["data_type"] = "fp16"
print(json.dumps(tao_deploy_specs, indent=4))

#### Run TRT Engine generation

In [ ]:
# Add --platform_id uuid for NVCF backend, where the uuid is a key from output of tao gpu-types
parent = job_map["export_" + model_name]
job_id = subprocess.getoutput(f"tao {model_name} create-job --kind experiment --action gen_trt_engine --name {model_name}_gen_trt_engine_job --encryption-key {encode_key} --workspace-id {workspace_id} --docker-env-vars '{json.dumps(docker_env_vars)}' --parent-job-id {parent} --specs '{json.dumps(tao_deploy_specs)}'")
job_map["gen_trt_engine_" + model_name] = job_id
print(job_id)
%store job_map

In [ ]:
# Check status (the file won't exist until the backend Toolkit container is running -- can take several minutes)
job_id = job_map["gen_trt_engine_" + model_name]
status = my_tail(model_name, job_id)

### TAO inference <a class="anchor" id="head-20"></a>

#### Provide TAO inference specs

In [ ]:
# Default inference model specs
tao_inference_specs_response = subprocess.getoutput(f"tao {model_name} get-job-schema --action inference")
tao_inference_specs_schema = json.loads(tao_inference_specs_response)
tao_inference_specs = tao_inference_specs_schema.get("default", {})
print(json.dumps(tao_inference_specs, indent=4))

In [ ]:
# Apply changes to specs if necessary
if model_name == "segformer":
    tao_inference_specs["dataset"]["segment"]["batch_size"] = 1
elif model_name == "mask2former":
    tao_inference_specs["model"]["sem_seg_head"]["num_classes"] = 90
    tao_inference_specs["dataset"]["contiguous_id"] = True
elif model_name == "mask_grounding_dino":
    tao_inference_specs["dataset"]["infer_data_sources"]["data_type"] = "OD"
print(json.dumps(tao_inference_specs, sort_keys=True, indent=4))

#### Run TAO inference

In [ ]:
# Add --platform_id uuid for NVCF backend, where the uuid is a key from output of tao gpu-types
parent = job_map["train_" + model_name]
train_datasets_json = json.dumps([train_dataset_id])
job_id = subprocess.getoutput(f"tao {model_name} create-job --kind experiment --action inference --name {model_name}_inference_job --encryption-key {encode_key} --workspace-id {workspace_id} --docker-env-vars '{json.dumps(docker_env_vars)}' --train-datasets '{train_datasets_json}' --eval-dataset {eval_dataset_id} --inference-dataset {eval_dataset_id} --parent-job-id {parent} --specs '{json.dumps(tao_inference_specs)}'")
job_map["tao_inference_" + model_name] = job_id
print(job_id)
%store job_map

In [ ]:
# Check status (the file won't exist until the backend Toolkit container is running -- can take several minutes)
job_id = job_map["tao_inference_" + model_name]
status = my_tail(model_name, job_id)

### TRT inference <a class="anchor" id="head-21"></a>

#### Provide TRT inference specs

In [ ]:
# Default inference model specs
trt_inference_specs_response = subprocess.getoutput(f"tao {model_name} get-job-schema --action inference")
trt_inference_specs_schema = json.loads(trt_inference_specs_response)
trt_inference_specs = trt_inference_specs_schema.get("default", {})
print(json.dumps(trt_inference_specs, indent=4))

In [ ]:
# Apply changes to specs if necessary
if model_name == "segformer":
    trt_inference_specs["dataset"]["segment"]["batch_size"] = 1
elif model_name == "mask2former":
    trt_inference_specs["model"]["object_mask_threshold"] = 0.0
elif model_name == "mask_grounding_dino":
    trt_inference_specs["dataset"]["batch_size"] = 1
    trt_inference_specs["dataset"]["infer_data_sources"]["data_type"] = "OD"

print(json.dumps(trt_inference_specs, sort_keys=True, indent=4))

#### Run TRT inference

In [ ]:
# Add --platform_id uuid for NVCF backend, where the uuid is a key from output of tao gpu-types
parent = job_map["gen_trt_engine_" + model_name]
train_datasets_json = json.dumps([train_dataset_id])
job_id = subprocess.getoutput(f"tao {model_name} create-job --kind experiment --action inference --name {model_name}_inference_job --encryption-key {encode_key} --workspace-id {workspace_id} --docker-env-vars '{json.dumps(docker_env_vars)}' --train-datasets '{train_datasets_json}' --eval-dataset {eval_dataset_id} --inference-dataset {eval_dataset_id} --parent-job-id {parent} --specs '{json.dumps(trt_inference_specs)}'")
job_map["trt_inference_" + model_name] = job_id
print(job_id)
%store job_map

In [ ]:
# Check status (the file won't exist until the backend Toolkit container is running -- can take several minutes)
job_id = job_map["trt_inference_" + model_name]
status = my_tail(model_name, job_id)

In [ ]:
# # Optional: Backup database with a mongodump file saved in workspace dump/archive/{backup_filename}
# backup_file_name = "mongodump.tar.gz" # FIXME 10
# subprocess.getoutput(f"tao {model_name} backup-workspace --workspace-id {workspace_id} --backup_file_name {backup_file_name}")

### Delete jobs <a class="anchor" id="head-22"></a>

In [ ]:
print("Deleting all created jobs...")

jobs_to_delete = []
for job_key, job_id in job_map.items():
    try:
        delete_response = subprocess.getoutput(f"tao {model_name} delete-job --job-id {job_id} --confirm")
        print(f"Deleted job: {job_key}")
    except Exception as e:
        print(f"Error deleting job {job_key}: {e}")

print(f"\n Job cleanup completed! Processed {len(jobs_to_delete)} jobs.")

### Delete dataset <a class="anchor" id="head-23"></a>

#### Delete train dataset

In [ ]:
! tao {model_name} delete-dataset --dataset-id {train_dataset_id}

#### Delete val dataset

In [ ]:
! tao {model_name} delete-dataset --dataset-id {eval_dataset_id}